In [1]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research', group='uni-basque-cou-1', project='main')

#mock backends and tools
from qiskit.test import mock
from qiskit.test.mock.backends import * #FakeVigo, FakeMontreal #FakeManhattan
from inspect import getmembers, isclass

""" Imports to Python functions """
import math
import array
import fractions
import numpy as np
import sys
import random
import time

import csv
import matplotlib.pyplot as plt

In [2]:
def encode(a,b,m,qR):
    if a == 1:
        m.x(qR[0])
    if b == 1:
        m.h(qR[0])
    return m
    
def decode(b,m,qR):
    if b == 1:
        m.h(qR[0]) #Deshacer hadamard
    return m

def agree(key1,key2,r,avg_diff,e=1):
    if len(key1)<2:
        return [False, "Key too short", key1, key2]
    diff = 0
    
    m = len(key1)
    t = math.ceil(m/r)
    selection = random.sample(range(t),t-1)
    
    for i in selection:
        if key1[i] == key2[i]:
            continue
        else:
            diff += 1
            
    #for i in selection:
        # these bits are public: discard them for the final key generation
    #    print(i)
    #    key1.pop(i)
    #    key2.pop(i)
    key1 = [v for i, v in enumerate(key1) if i not in selection]
    key2 = [v for i, v in enumerate(key2) if i not in selection]
        
    if diff > e*avg_diff:
        state = False
    else:
        state = True
            
    return [state, diff, key1, key2]

def bb84_shot(a_A,EVE,server):
    qm = QuantumRegister(1)
    cm = ClassicalRegister(1)

    m_A  = QuantumCircuit(qm,cm)
    m_B  = QuantumCircuit(qm,cm)
    
    b_A = np.random.randint(0,2)
    b_B = np.random.randint(0,2)

    encode(a_A,b_A,m_A,qm)
    
    if EVE==1:
        m_E  = QuantumCircuit(qm,cm)
        b_E  = np.random.randint(0,2)
        m_E = m_A
    
        decode(b_E,m_E,qm)
        m_E.measure(qm,cm)
        m_E_exec = execute(m_E,server,shots=n_shots).result().get_counts(m_E)
        a_E = int(list(m_E_exec.keys())[0])
        m_A = encode(a_E,b_E,m_E,qm)
        
    m_B = m_A
    decode(b_B,m_B,qm)
        
    m_B.measure(qm,cm)
    m_B_exec = execute(m_B,server,shots=n_shots).result().get_counts(m_B)
    a_B=int(list(m_B_exec.keys())[0])
    
    return a_B,b_A,b_B

def bb84_qcom(n,Eve,server):
    a_Alice = []
    a_Bob   = []
    b_Alice = []
    b_Bob   = []
    for i in range(n):
        a_A = np.random.randint(0,2)
        comm = bb84_shot(a_A,Eve[i],server)
        a_Alice.append(a_A)
        a_Bob.append(comm[0])
        b_Alice.append(comm[1])
        b_Bob.append(comm[2])
        
    return[a_Alice,a_Bob,b_Alice,b_Bob]

def bb84(N,Eve,avg_diff,server,r=3):
    key_Alice = []
    key_Bob   = []

    # QUANTUM COMMUNICATION PHASE
    qcom_result = bb84_qcom(N,Eve,server)

    a_Alice = qcom_result[0]
    a_Bob   = qcom_result[1]
    b_Alice = qcom_result[2]
    b_Bob   = qcom_result[3]

    # AGREEMENT PHASE: compare bases
    for i in range(len(a_Bob)):
        if b_Bob[i] == b_Alice[i]:
            key_Alice.append(a_Alice[i])
            key_Bob.append(int(a_Bob[i]))

    # AGREEMENT PHASE: compare bits
    final = agree(key_Alice,key_Bob,r,avg_diff,1)

    #if not final[0]:
    #    print("Eavesdropper detected!")

    return final #[state, diff, key1, key2]

def calibrate(N_test,server,r=3):
    Eve_test = N_test*[0]

    avg = 0
    for i in range(10):
        test = bb84(N_test,Eve_test,0,server,r)
        if test[0]:
            avg += test[1]

    avg = 0.1*avg/N_test
    return avg

In [7]:
def test_r(server,n_eve,Eve,r_list):
    for r in r_list:
        data=open("BB84_COMP_"+str(server)+"_"+str(n_eve)+"_"+str(r)+".txt",mode)

        data.write(str(server))
        data.write('\n'+str(r))
        data.write('\nEVE: {0}'.format(str(Eve)))

        avg_diff = calibrate(math.floor(nq/3),simulator,r)

        #print("\nAverage disagreement in keys with no Eve: {0}.".format(avg_diff))
        data.write("\nAverage disagreement in keys with no Eve: {0}.\n".format(avg_diff))

        failure_list = []
        success_list = []
        for i in range(tries):
            result_bb84 = bb84(nq,Eve,avg_diff,server,r)
            if result_bb84[0]:
                success_list.append(result_bb84[2])
                success_list.append(result_bb84[3])
            else:
                failure_list.append(result_bb84[1])

        #print("The protocol succeeded in {0}% of cases.".format(len(success_list)/tries*50))
        data.write("{0}%\n".format(len(success_list)/tries*50))
        for key in success_list:
            for i in key:
                data.write(str(i))
            data.write('\n')
            
    data.close()
    
def extract_data_r(server,n_eve,r_list):
    mode = "r"
    data_matrix = []

    for i_r in range(len(r_list)):
        data=open("BB84_COMP_"+str(server)+"_"+str(n_eve)+"_"+str(r_list[i_r])+".txt",mode)
        content = data.readlines()

        server_data = content[0][:-1]
        r_data      = content[1][:-1]
        Eve_data    = content[2][:-1]
        diff_data   = content[3][:-1]
        valid_rate  = content[4][:-1]

        content = content[5:]

        print("R:",r_data)
        print(valid_rate)

        if len(content) < 1: # If file has no keys, skip calculations
            print("No keys found!\n")
            continue

        rate = 0
        len_list = []

        for l in range(0,len(content)-1,2):
            line_1 = list(content[l])[:-1]
            line_2 = list(content[l+1])[:-1]

            len_list.append(len(line_1))

            correct = 0
            incorrect = 0
            for i in range(len(line_1)):
                if line_1[i] == line_2[i]:
                    correct += 1
                else:
                    incorrect += 1
            rate = rate+(correct/len(line_1))

        avg_rate = rate/len(len_list)
        avg_len  = sum(len_list)/len(len_list)
        #print("Average key agreement rate:",avg_rate*100)
        #print("Average key length:",avg_len)
        #print("\n")
        data_matrix.append([server_data,nq,n_eve,valid_rate,avg_len,avg_rate])

    # create csv file 
    with open('BB84_ALT_'+str(n_eve)+'agree.csv', mode='w+') as qkd_file:
        qkd_writer = csv.writer(qkd_file, delimiter=',', quotechar='"')
        qkd_writer.writerow(["Server","Number of qubits","Number of Eve","Eve not detected","Average key length","Average correct % of key"])
        qkd_writer.writerows(data_matrix)

In [4]:
simulator = Aer.get_backend('qasm_simulator')
santiago  = provider.get_backend('ibmq_santiago')
bogota    = provider.get_backend('ibmq_bogota')

fake_bogota = FakeBogota()
n_shots = 1 # CHANGE TO EMULATE POSSIBLE PNS ATTACK?

server = simulator

In [9]:
nq = server.configuration().n_qubits
tries = 100
mode = "w+"

r_list = range(2,6)

for k in range(19,nq): # Eve incrreases her number of interventions...
    Eve = np.array([1]*k + [0]*(nq-k))
    np.random.shuffle(Eve)
    print("{0} eavesdroppers.".format(k))
    test_r(simulator,k,Eve,r_list)
    extract_data_r(simulator,k,r_list)

19 eavesdroppers.
R: 2
10.0%
R: 3
26.0%
R: 4
32.0%
R: 5
39.0%
20 eavesdroppers.
R: 2
8.0%
R: 3
24.0%
R: 4
27.0%
R: 5
44.0%
21 eavesdroppers.
R: 2
6.0%
R: 3
11.0%
R: 4
18.0%
R: 5
27.0%
22 eavesdroppers.
R: 2
5.0%
R: 3
17.0%
R: 4
31.0%
R: 5
45.0%
23 eavesdroppers.
R: 2
9.0%
R: 3
15.0%
R: 4
22.0%
R: 5
33.0%
24 eavesdroppers.
R: 2
4.0%
R: 3
8.0%
R: 4
21.0%
R: 5
24.0%
25 eavesdroppers.
R: 2
4.0%
R: 3
9.0%
R: 4
20.0%
R: 5
9.0%
26 eavesdroppers.
R: 2
1.0%
R: 3
4.0%
R: 4
13.0%
R: 5
15.0%
27 eavesdroppers.
R: 2
4.0%
R: 3
20.0%
R: 4
19.0%
R: 5
37.0%
28 eavesdroppers.
R: 2
0.0%
No keys found!

R: 3
7.000000000000001%
R: 4
18.0%
R: 5
18.0%
29 eavesdroppers.
R: 2
1.0%
R: 3
7.000000000000001%
R: 4
11.0%
R: 5
21.0%
